In [1]:
from diff_diffusion.tlm import jacobian_tlm
from diff_diffusion.tlm.rhs_tlm import TLM
from diff_diffusion.analysis.plot import plot_distributions_tlm
from diff_diffusion.solver.total_current import TotalCurrent
from scipy.integrate import solve_ivp
from diff_diffusion.utils.radauDAE import RadauDAE, solve_ivp_custom
from matplotlib import pyplot as plt

import numpy as np
from diff_diffusion.utils import parameters, grid
from diff_diffusion.tlm import mass_tlm
from diff_diffusion.solver import initial_conditions

In [2]:
params = parameters.Params(slm=False, test_case_slm=False, DI=None)
grids = grid.Grid(params)
ic = initial_conditions.Initial_Conditions(params, grids)
mass = mass_tlm.Mass(params, grids)
jac = jacobian_tlm.Jac(params)
total_current = TotalCurrent(params, grids)

sol_init = ic.sol_init_tlm
mass = mass.M
jac = jac.jac

method = RadauDAE
rtol = params.rtol
atol = params.atol

tf = 1
tlm_func = TLM(params)

Calculating consistent initial conditions.
Complete


In [3]:
dae_fun = lambda t, u: tlm_func(t, u, 'pbi')

sol = solve_ivp(fun=dae_fun, t_span=(0.0, tf), y0=sol_init,
                rtol=rtol, atol=atol, jac_sparsity=jac,
                method=method, vectorized=False, dense_output=False,
                bPrintProgress=True, mass_matrix=mass,
                scale_residuals=True,
                factor_on_non_convergence=0.95
                )
# float64 limitation = 2.2E-308 to 1.7E+308.

  0%|                                                                                                                                             | 00:00>?

KeyboardInterrupt: 

In [ ]:
10 * np.abs(np.nextafter(0, np.inf) - 0)

In [ ]:
t = sol.t
dae_fun = lambda t, u: tlm_func(t, u, 'test1')
sol_init = sol.y[:,-1]

sol = solve_ivp(fun=dae_fun, t_span=(0.0, 10*tf), y0=sol_init,
            rtol=rtol, atol=atol, jac_sparsity=jac,
            method=method, vectorized=False, dense_output=True,
            bPrintProgress=True, mass_matrix=mass
            )

In [ ]:
sol_init = sol.y[:,-1]
tf_reverse_scan = params.tf_scan
dae_fun = lambda t, u: tlm_func(t, u, 'reverse_scan')

sol = solve_ivp_custom(fun=dae_fun, t_span=(0.0, tf_reverse_scan), y0=sol_init,
                rtol=rtol, atol=atol, jac_sparsity=jac,
                method=method, vectorized=False, dense_output=True,
                bPrintProgress=True, mass_matrix=mass,
                factor_on_non_convergence=0.5
                )